# 발표 소단원 리스트

7.12
8.3
8.6
8.9
8.12
8.16
8.20
8.24

# 전체 요약자료

-


## 8.3 context manager class

In [ ]:
# context manager handles resources in preservative manner
# generally, context manager should save resource regardless of exceptions
# mostly this is implemented as below

try:
    opened_file = open('some-file')
    r = opened_file.read()
    # do somethong meaningful
else:
    opened_file.close()

In [7]:
from socket import socket, AF_INET, SOCK_STREAM

# __enter__ and __exit__ method each is called at the beginning and the end of with statement
# those are critical to implement context manager class
class SocketConnector:
    def __init__(self, ip):
        self.ip = ip
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []
    def __enter__(self):
        # the return value of this function will bind to as statement
        sock = socket(self.family, self.type)
        sock.connect(self.ip)
        self.connections.append(sock)
        return sock
    def __exit__(self, *args):
        self.connections.pop().close()

with SocketConnector(('localhost',80)) as s:
    # do somethong meaningful
    pass
    
        

## 8.6 property decorator

In [48]:
class Man:
    def __init__(self, name):
        self.name = name
    
    @property
    def name(self):
        return self._name
    
    @name.setter # this decorator is defined after establishing @property ahead
    def name(self, value):
        if type(value) != str:
            raise TypeError
        self._name = value

man = Man('minkyu') # fine
man.name = 'Deoksoo' # fine
man.name = 10000 # TypeError

## 8.9 descriptor class

In [43]:
class Descriptor:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, inst, cls):
        if inst is None:
            return self
        else:
            # do something meaningful
            return inst.__dict__[self.name]
    
    def __set__(self, inst, val):
        if val == 'describe':
            print('Descriptor!!')
            return
        inst.__dict__[self.name] = val
    
    def __delete__(self, inst):
        # do something meaningful
        del inst.__dict__[self.name]
        

class SomeClass:
    f = Descriptor('foo') 
    # class attr f, is now the descriptor that maps to instance attr f
    # value that refers instance attr f, will be held in that instance by the key 'foo'
    # (probably) this attr name and key shall not be different
    b = Descriptor('bar')
    x = Descriptor('x')
    
    def __init__(self, ff, bb, cc):
        self.f = ff
        self.b = bb
        self.c = cc

s = SomeClass(1,2,3)
print(SomeClass.f) # <__main__.Descriptor object at 0x00D42ED0>
print(s.f) # 1
print(s.__dict__) # {'foo': 1, 'bar': 2, 'c': 3}
s.f = 'describe' # Descriptor!!
s.b = 'describe' # Descriptor!!
s.x = 'describe' # Descriptor!!
s.c = 'describe' # nothing happens, since instance attr c, self.c, is not assigned to descriptor
print(s.__dict__) # {'foo': 1, 'bar': 2, 'c': 'describe'}

1
{'foo': 1, 'bar': 2, 'c': 3}
Descriptor!!
Descriptor!!
Descriptor!!
{'foo': 1, 'bar': 2, 'c': 'describe'}


## 8.12  ABC; Abtract Base Class for creating interface

In [64]:
from abc import ABCMeta, abstractmethod


class Interface(metaclass=ABCMeta):
    @abstractmethod
    def foo(self, param=1):
        pass
    
    @abstractmethod
    def bar(self):
        pass

    
class Child(Interface):
    def foo(self, param=1):
        print('hello', param)
    
    def bar(self):
        print('world')
    
    def hoi(self):
        print('python!')

i = Interface() # TypeError: you cannot directly instantiate ABC
c = Child() # if any of abstractmethod omitted, error is raised. 
c.foo()
c.bar()
c.hoi()

# below does not show error
# you should use ABCMeta and @abstractclass both to implement interface
class Parent:
    @abstractmethod
    def foo():
        pass

class C(Parent):
    def a():
        pass

hello 1
world
python!


## 8.16 classmethod decorator for implementing alternate constructor

In [78]:
import time

class Date:
    def __init__(self, m, d, y):
        self.m = m
        self.d = d
        self.y = y
    
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_mon, t.tm_mday, t.tm_year)


d = Date(1,31,2019)
dd = Date.today()
print(d, d.__dict__)
print(dd, dd.__dict__)

class NewDate(Date): # this way, inheritance is easy with @classmethod
    def foo(self, param):
        print(param)

ddd = NewDate.today()
print(ddd, ddd.__dict__)

<__main__.Date object at 0x013446B0> {'m': 1, 'd': 31, 'y': 2019}
<__main__.Date object at 0x01344690> {'m': 5, 'd': 5, 'y': 2019}
<__main__.NewDate object at 0x01549F90> {'m': 5, 'd': 5, 'y': 2019}


## 8.20 call method by string

In [80]:
n = NewDate.today()
f = getattr(n, 'foo')
f('hello world')

hello world


## 8.24 class comparison 

In [82]:
from functools import total_ordering

@total_ordering
class Test:
    def __init__(self, param):
        self.param = param
    
    def __eq__(self, other):
        return self.param == other.param

    def __lt__(self, other):
        return self.param < other.param

t1 = Test(1)
t2 = Test(2)
print(t1<t2)

True
